In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [ ]:
train=pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv', nrows=3000)

In [ ]:
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/jane-street-market-prediction/example_test.csv')

In [ ]:
test.shape

In [ ]:
test.describe()

In [ ]:
train.describe()

In [ ]:
sample=pd.read_csv('/kaggle/input/jane-street-market-prediction/example_sample_submission.csv')

In [ ]:
sample

In [ ]:
df1=train
X = df1.loc[:, df1.columns.str.contains('feature')]
y = (df1['resp'] > 0)*1 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .25, random_state = 69)

In [ ]:
imp = SimpleImputer(missing_values=np.nan , strategy='mean')
ss = StandardScaler()

X_train = imp.fit_transform(X_train)
X_test = imp.transform(X_test)
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train_scaled,y_train)
print('Log Reg Score: {:.3f}'.format(logreg.score(X_test_scaled,y_test)))

In [ ]:
import janestreet
import tqdm
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

for (test_df, sample_prediction_df) in tqdm.tqdm(iter_test):
    y_preds = []
    X_test_1 = test_df.loc[:, test_df.columns.str.contains('feature')]
    X_test_1 = ss.transform(imp.transform(X_test_1))
    sample_prediction_df.action = logreg.predict(X_test_1.reshape(1,-1))[0]
    env.predict(sample_prediction_df)